In [22]:
!dvc run --force -n gen_env -w .. -d data/default_env.pickle -p true_norm,colour_specific,shape_specific,target_area -o data/env.pickle python scripts/gen_env.py

In [27]:
!dvc run --force -n gen_observations -w .. -d data/env.pickle -p target_area,true_norm,num_observations,obs_data_set -o data/observations.pickle python scripts/gen_observations.py

Running stage 'gen_observations' with command:                        core>
	python scripts/gen_observations.py
Adding stage 'gen_observations' in 'dvc.yaml'                         core>
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock


In [33]:
!dvc run --force -n gen_mcmc_chains -w .. -d data/env.pickle -d data/observations.pickle -p n,m,rf,colour_specific,shape_specific,target_area -o data/chains.pickle --plots chain_likelihoods.csv python scripts/gen_mcmc_chains.py

------------------------------------------------
                NORM Number=2
------------------------------------------------
   > PERMISSION: action putdown
     for colour b, shape any and zone 2


===========================Iteration=925===========================
Old Expression: Log-Likelihood=-54410.076496309426
New Expression: Log-Likelihood=-54410.076496309426
log(factor) = 0
log(p_accept)=0.0000000
E_new accepted
------------------------------------------------
                NORM Number=1
------------------------------------------------
   > OBLIGATION:
     put in zone 3 if handling colour any and shape any
     and history is 
      ['Moved', ['Colour', 'r'], ['Shape', 'circle'], ['Zone', '3']]
      ['Moved', ['Colour', 'b'], ['Shape', 'any'], ['Zone', '1']]
------------------------------------------------
                NORM Number=2
------------------------------------------------
   > PERMISSION: action putdown
     for colour b, shape any and zone 2


==============

In [ ]:
!dvc repro -f